In [ ]:
!pip install -r reqt.txt

In [ ]:
!pip install mysql-connector-python transformers torch accelerateb

In [ ]:
!pip install mysql-connector-python

In [ ]:
from huggingface_hub import login

login(token={{tocken_id}})

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "codellama/CodeLlama-7b-Instruct-hf"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

In [ ]:
def generate_response(prompt, max_length=200):
    """
    Generates a response from Llama 3.1-8B based on the given prompt.

    :param prompt: The input text or question.
    :param max_length: Maximum length of the generated response.
    :return: The generated response as a string.
    """
    input_text = f"### Instruction:\n{prompt}\n### Response:"

    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=max_length)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response.split("### Response:")[-1].strip()


In [ ]:
user_prompt = "Explain the concept of recursion in simple terms."
response = generate_response(user_prompt)
print("Llama 3 Response:", response)

In [ ]:
import torch
print("Is CUDA Available? ->", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# model_name = "codellama/CodeLlama-7b-Instruct-hf"
# model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"
model_name = "defog/sqlcoder-7b-2"

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model onto GPU if available
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,  # Use float16 for GPU, float32 for CPU
    device_map="auto" if torch.cuda.is_available() else {"": "cpu"}  # Auto-assign layers if GPU available
)

def generate_response(prompt, max_length=200):
    """
    Generates a response from Llama 3.1-8B based on the given prompt.

    :param prompt: The input text or question.
    :param max_length: Maximum length of the generated response.
    :return: The generated response as a string.
    """
    input_text = f"### Instruction:\n{prompt}\n### Response:"

    # Ensure inputs are moved to GPU if available
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=max_length)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response.split("### Response:")[-1].strip()

# Example Usage
user_prompt = "Explain the concept of recursion in simple terms."
response = generate_response(user_prompt)

print("Llama 3 Response:", response)

In [ ]:
import mysql.connector

# Replace with your RDS details
db_config = {
    "host": "database-1.c54yoa4ekvd5.eu-north-1.rds.amazonaws.com",
    "user": "admin",
    "password": "mypass.awsdb",
    "database": "ecommerce"
}

def fetch_data(query):
    try:
        connection = mysql.connector.connect(**db_config)
        cursor = connection.cursor()
        cursor.execute(query)
        result = cursor.fetchall()
        cursor.close()
        connection.close()
        return result
    except mysql.connector.Error as err:
        return f"Error: {err}"

In [ ]:
def generate_sql_query(prompt):
    input_text = f"### Instruction: Generate an SQL query for the following request.\n{prompt}\n### SQL:"
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=200)

    query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return query.split("### SQL:")[-1].strip()

In [ ]:
def generate_sql(query, max_length=1200):
    """
    Generates an SQL query based on the user's request and the given database schema.

    :param query: The natural language request for an SQL query.
    :param max_length: Maximum length of the generated SQL query.
    :return: The generated SQL query as a string.
    """
    prompt = f"{DB_SCHEMA}\n### Task: Generate an MySQL query(I am using MySQL server version) for the following request.\nRequest: {query}\n### SQL Query:\n"

    # Ensure inputs are moved to GPU if available
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=max_length)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response.split("### SQL Query:")[-1].strip()

In [ ]:
user_request = "Get the name, email, phone of all users where orders are completed"
sql_query = generate_sql(user_request)

print("\nGenerated SQL Query:\n", sql_query)

data = fetch_data(sql_query)
print("Query Result:", data)

In [ ]:
user_request = "Get the deatils of users who have purchased iPhone 13"
sql_query = generate_sql_query(user_request)
print("Generated SQL Query:", sql_query)

# Fetch data from MySQL
data = fetch_data(sql_query)
print("Query Result:", data)

In [ ]:
DB_SCHEMA = """
You are an AI SQL expert. Below is the MySQL database schema.

---
### Database Schema (MySQL Server):
#### `users` (Stores user details)
- `id` BIGINT PRIMARY KEY AUTO_INCREMENT
- `name` VARCHAR(100) NOT NULL
- `email` VARCHAR(100) UNIQUE NOT NULL
- `phone` VARCHAR(15)
- `address` TEXT
- `created_at` TIMESTAMP DEFAULT CURRENT_TIMESTAMP

#### `categories` (Stores product categories)
- `id` BIGINT PRIMARY KEY AUTO_INCREMENT
- `name` VARCHAR(100) NOT NULL
- `description` TEXT
- `created_at` TIMESTAMP DEFAULT CURRENT_TIMESTAMP

#### `products` (Stores product details)
- `id` BIGINT PRIMARY KEY AUTO_INCREMENT
- `name` VARCHAR(100) NOT NULL
- `description` TEXT
- `price` DECIMAL(10,2) NOT NULL
- `stock_quantity` INT DEFAULT 0
- `category_id` BIGINT NOT NULL
- `created_at` TIMESTAMP DEFAULT CURRENT_TIMESTAMP
- FOREIGN KEY (`category_id`) REFERENCES `categories`(`id`) ON DELETE CASCADE
- INDEX (`category_id`)

#### `orders` (Stores order details)
- `id` BIGINT PRIMARY KEY AUTO_INCREMENT
- `user_id` BIGINT NOT NULL
- `order_date` TIMESTAMP DEFAULT CURRENT_TIMESTAMP
- `total_amount` DECIMAL(10,2) NOT NULL
- `status` ENUM('Pending', 'Completed', 'Cancelled') DEFAULT 'Pending'
- FOREIGN KEY (`user_id`) REFERENCES `users`(`id`) ON DELETE CASCADE
- INDEX (`user_id`)

#### `order_items` (Stores products in an order)
- `id` BIGINT PRIMARY KEY AUTO_INCREMENT
- `order_id` BIGINT NOT NULL
- `product_id` BIGINT NOT NULL
- `quantity` INT CHECK (`quantity` > 0)
- `price` DECIMAL(10,2) NOT NULL
- FOREIGN KEY (`order_id`) REFERENCES `orders`(`id`) ON DELETE CASCADE
- FOREIGN KEY (`product_id`) REFERENCES `products`(`id`) ON DELETE CASCADE
- INDEX (`order_id`), INDEX (`product_id`)

#### `payments` (Stores payment details)
- `id` BIGINT PRIMARY KEY AUTO_INCREMENT
- `order_id` BIGINT NOT NULL
- `payment_date` TIMESTAMP DEFAULT CURRENT_TIMESTAMP
- `amount` DECIMAL(10,2) NOT NULL
- `payment_method` ENUM('Credit Card', 'PayPal', 'Bank Transfer', 'Cash')
- `status` ENUM('Pending', 'Completed', 'Failed') DEFAULT 'Pending'
- FOREIGN KEY (`order_id`) REFERENCES `orders`(`id`) ON DELETE CASCADE
- INDEX (`order_id`)

---
### ** SQL Query Rules:**
1️ **Use correct `JOINs`** based on Foreign Key relationships.
2️ **Alias tables** (`u`, `o`, `p`, etc.) for readability.
3️ **Ensure all selected columns exist** in the schema.
4️ **Use `WHERE`, `GROUP BY`, and `HAVING` properly** to filter results.
5️ **For date-based queries**, use `NOW()`, `INTERVAL`, and `DATE_FORMAT()`.
6️ **Use `COUNT()`, `SUM()`, and `AVG()` carefully in aggregation queries.**
7 **For case-insensitive searches in MySQL, always use `LOWER(column) LIKE '%value%'` instead of `ILIKE`.**
8 **Use `WHERE`, `GROUP BY`, and `HAVING` properly** to filter results.

---
### ** Example Queries**
####  Get total orders per user in the last 30 days:
```sql
SELECT u.id, u.name, COUNT(o.id) AS total_orders
FROM users u
LEFT JOIN orders o ON u.id = o.user_id
WHERE o.order_date >= NOW() - INTERVAL 30 DAY
GROUP BY u.id, u.name
ORDER BY total_orders DESC;
"""

In [ ]:
DB_SCHEMA = """
You are an expert SQL query generator. Below is the MySQL database schema.
CREATE TABLE users (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100),
    email VARCHAR(100) UNIQUE,
    phone VARCHAR(15),
    address TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE categories (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100),
    description TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE products (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100),
    description TEXT,
    price DECIMAL(10, 2),
    stock_quantity INT,
    category_id INT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (category_id) REFERENCES categories(id)
);

CREATE TABLE orders (
    id INT AUTO_INCREMENT PRIMARY KEY,
    user_id INT,
    order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    total_amount DECIMAL(10, 2),
    status VARCHAR(20),
    FOREIGN KEY (user_id) REFERENCES users(id)
);

CREATE TABLE order_items (
    id INT AUTO_INCREMENT PRIMARY KEY,
    order_id INT,
    product_id INT,
    quantity INT,
    price DECIMAL(10, 2),
    FOREIGN KEY (order_id) REFERENCES orders(id),
    FOREIGN KEY (product_id) REFERENCES products(id)
);

CREATE TABLE payments (
    id INT AUTO_INCREMENT PRIMARY KEY,
    order_id INT,
    payment_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    amount DECIMAL(10, 2),
    payment_method VARCHAR(50),
    status VARCHAR(20),
    FOREIGN KEY (order_id) REFERENCES orders(id)
);
"""

In [ ]:
DB_SCHEMA = """
You are an expert SQL query generator. Below is the MySQL database schema.

---
### Database Schema:
#### `users` (Stores user details)
- `id` INT (Primary Key)
- `name` VARCHAR(100)
- `email` VARCHAR(100) (Unique)
- `phone` VARCHAR(15)
- `address` TEXT
- `created_at` TIMESTAMP (Default: CURRENT_TIMESTAMP)

#### `categories` (Stores product categories)
- `id` INT (Primary Key)
- `name` VARCHAR(100)
- `description` TEXT
- `created_at` TIMESTAMP (Default: CURRENT_TIMESTAMP)

#### `products` (Stores product details)
- `id` INT (Primary Key)
- `name` VARCHAR(100)
- `description` TEXT
- `price` DECIMAL(10, 2)
- `stock_quantity` INT
- `category_id` INT (Foreign Key, references `categories.id`)
- `created_at` TIMESTAMP (Default: CURRENT_TIMESTAMP)

#### `orders` (Stores order details)
- `id` INT (Primary Key)
- `user_id` INT (Foreign Key, references `users.id`)
- `order_date` TIMESTAMP (Default: CURRENT_TIMESTAMP)
- `total_amount` DECIMAL(10, 2)
- `status` VARCHAR(20)

#### `order_items` (Stores products in an order)
- `id` INT (Primary Key)
- `order_id` INT (Foreign Key, references `orders.id`)
- `product_id` INT (Foreign Key, references `products.id`)
- `quantity` INT
- `price` DECIMAL(10, 2)

#### `payments` (Stores payment details)
- `id` INT (Primary Key)
- `order_id` INT (Foreign Key, references `orders.id`)
- `payment_date` TIMESTAMP (Default: CURRENT_TIMESTAMP)
- `amount` DECIMAL(10, 2)
- `payment_method` VARCHAR(50)
- `status` VARCHAR(20)

---
### **Important Table Relationships:**
1. **Users & Orders:**
   - `orders.user_id` links to `users.id`
2. **Orders & Products:**
   - `order_items.order_id` links to `orders.id`
   - `order_items.product_id` links to `products.id`
3. **Products & Categories:**
   - `products.category_id` links to `categories.id`

---
### **Rules for SQL Query Generation:**
1. **Always use the correct foreign key relationships.**
2. **Never use `o.product_id`, instead, use `oi.product_id`.**
3. **Join `order_items` to `products` when filtering by product name.**
4. **Join `products` to `categories` when aggregating by category.**
5. **Ensure all column names exist in the schema before using them.**

---
### **Example Queries:**
#### Get users who purchased a specific product:
```sql
SELECT u.id, u.name, u.email, o.order_date
FROM users u
JOIN orders o ON u.id = o.user_id
JOIN order_items oi ON o.id = oi.order_id
JOIN products p ON oi.product_id = p.id
WHERE p.name = 'iPhone 13'
ORDER BY o.order_date DESC;
"""